***

* [Outline](../0_Introduction/0_introduction.ipynb)
* [Glossary](../0_Introduction/1_glossary.ipynb)
* [7. Observing Systems](0_introduction.ipynb)  
    * Previous: [7.2 The Radio Interferometer Measurement Equation (RIME)](7_2_rime.ipynb)  
    * Next: [7.4 Digital Electronics](7_4_digital.ipynb)

***

Import standard modules:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import HTML 
HTML('../style/course.css') #apply general CSS

In [ ]:
HTML('../style/code_toggle.html')

***

**TODO**

* introduce why this is important, connect as a direction independent effect
* clarify what gain is: is it an amplitude or amp and phase
* explain system gain
* define: amplification, attenuation, intergation time, bandwidth
* brief review of decibels
* radiometer equation, noise (system and sky), define CMB
* in tsys: define spillover, give approximate ranges of values
* say tsys is the noisiness of the instrument/measurement system
* sensitivity: example, setup an observation
* SEFD https://science.nrao.edu/facilities/vla/docs/manuals/oss/performance/sensitivity
* analogue signal chain diagram
* receiver/feed, balun, wave guide (image)
* OMT, cryostat (image)
* LNA (image)
* cables (image)
* bandpass, notch, lowpass, highpass, matched filters (image)
* example: typical bandpass (kat-7)
* example: gain variation for a set of antennas (kat-7)
* example: imag/vis waterfall with and without gain variations
* hetrodyne systems, mixers, local oscilators
* ADCs, the final analogue component, dynamic range and clipping (block diagram)
* reiterate what system gain is

## 7.3 Analogue Electronics (G- and B-Jones)

The system electronics accounts for most of the variation and noise during an observation. And hence, are the most basic component of the RIME which must be properly modelled and, as we will see later, calibrated to recover the original signal. The system electronics encompasses all analogue and digital components which are used to to convert the electromagnetic signal to a conditioned voltage (via amplifiers and filters) and then compute the correlation between antennas (via FFT and multiplication) to produce the digital visibility. This system 'gain', denoted as $G$ in the RIME, has both a time and frequency-dependent effect on the signal.

### 7.3.1 Analogue Receiver Front-End

Electro-magnetic waves are converted to a current via the antenna feed (using a balun or waveguide), from there the signal can be treated as a voltage in an electronic circuit. But, the signal is very weak, and can be treated as approximately Gaussian noise. That noise is what we would like to measure, as that noise is from sources in the sky. But, there are other types of noise, which we don't want, leaking into the measurement. At the earliest possible point in the analogue receiver chain the signal must be amplified using a *low-noise amplifier*. To understand why this amplification is necessary we must start with the *system temperature*.

The sky signal, if we consider it to be a voltage in an electrical circuit, is a weak Johnson-Nyquist noise source, also known as a 'cold' source. Johnson-Nyquist noise quantifies the power of a voltage signal in units of Kelvin.

$$P=k_B T \Delta \nu \label{eq:johnson_noise}$$

Where $P$ is the power of the voltage signal, $k_B$ is the Boltzmann constant, $T$ is temperature in Kelvin, and $\Delta \nu$ is the bandwidth of the signal in Hertz. Thus, in literature the noise power of a radio telescope is often described in terms of a temperature

$$T = \frac{P}{k_B \Delta \nu} \label{eq:johnson_noise_rearange}$$

At L-band (1.4 GHz) the sky temperature is approximately 10 K, 2.73 K of which are from the CMB. The temperature of the sky varies depending on the observing frequency, for example at low frequency, say 100 MHz, the sky is close to 1000 K. The Earth is approximated as a 300 K black body. We can see that if even a small percentage of the ground is picked up in the receiver then the sky signal can be buried underneath, especially at higher frequencies. The design of the early stages of the analogue receiver chain must pay important attention to which components are used, as each one adds to the system temperature. There is additional noise from ground pickup, the atmosphere (depending on observing frequency), the antenna feed, filters, amplifier, and passive electronic components.

A low-noise amplifier is used to boost a very weak signal typically by a factor of 1000 (30 dB) or 10000 (40 dB) while introducing minimal noise. An LNA is introduced as earlier as possible into the analogue chain to have the effect of boosting the sky signal, so as it proceeds through the rest of the system, it remains the dominant signal. The system temperature is of the form

$$ T_{\textrm{sys}} = T_{\textrm{sky}} + T_{\textrm{atmosphere}} + T_{\textrm{spillover}} + T_{\textrm{rx}} + \ldots \label{eq:system_temp}$$

where $T_{\textrm{sky}}$ is the noise contribution from the sky which we are most interested in. But, there is additional noise from the atmosphere, ionosphere, ground pick-up of the telescope, and the receiver system which is of the form

$$ T_{\textrm{rx}} = T_{\textrm{feed}} + \frac{T_{\textrm{passive}}}{G_{\textrm{feed}}} + \frac{T_{\textrm{LNA}}}{G_{\textrm{feed}} G_{\textrm{passive}}} + \frac{T_{\textrm{amp}}}{G_{\textrm{feed}} G_{\textrm{passive}} G_{\textrm{LNA}}} + \ldots \label{eq:rx_temp}$$

where the $T$ terms are the additional noise temperatures introduced with each component, and $G$ is the 'gain' term which can be less than 1 (attenuation) or greater than 1 (amplification). The gain from the feed and passive components will be less than 1, and are similar to an efficiency term in when discussing the primary beam, these will cause the temperature from these components to increase in the system temperature. By having the LNA as early as possible we see that the weak sky signal is amplified, and noisier components such as filters and second stage amplifiers can be introduced without introducing significant noise compared to the sky signal. The LNA and the front-end analogue components are often cooled using a heat pump or a *cryostat*, a sealed vacuum container which can be cooled down to $\sim 10$ Kelvin to reduce the system noise. For low-frequency systems the sky is 'hotter' than room temperature, so additional cooling offers minimal improvements in the system temperature.

### 7.3.2 Sensitivity and the Radiometer Equation

Why would we like our system to be as 'cold' as possible? The classic equation to describe the sensitivity of a radio antenna is the ideal *radiometer equation*.

$$
\sigma_{T} = \frac{T_{\textrm{sys}}}{\sqrt{\Delta\nu \tau}}
\label{eq:rediometer}
$$

which describes the necessary bandwidth ($\Delta \nu$) and integration time ($\tau$) required to reach a desired noise level ($\sigma_{T}$) for an ideal system with a given system temperature ($T_{\textrm{sys}}$) when observing a broadband source. For example, If there are two systems, with the same bandwidth $\Delta \nu$, in which one has a system temperature twice that of the other, i.e. $T_{\textrm{sys,0}} = 2 T_{\textrm{sys,1}}$, then $T_{\textrm{sys,0}}$ will take 4 times longer to reach the same sensitivity as $T_{\textrm{sys,1}}$.

The radiometer equation has been presented for an ideal system, but in reality analogue systems are unstable and introduce 'gain fluctuations'. These fluctuations come about due to a number of effects: the physical temperature of the electronics, cross-coupling leakage, RF sheilding among others. The stability of the system during an observation will depend on how well these effects are managed. These gain fluctuations will have both a time and frequency dependence.

### 7.3.3 The Bandpass

The gain fluctuations of the electronics systems can often be seperated into two components. The time-stable frequency dependent component encapsulated in the *bandpass response*. The frequency-dependent structure of the sky is modified by the optics, receiver, analogue components, and the digital signal processing. Generally, a telescope will have a frequency-dependent efficiency which have the effect of decreasing gain as the frequency increases. The feed will be tuned for a specific response, which can even introduce sharp features into the bandpass. Like the feed, the LNA will be tuned for a specific frequency response. Importantly, in *heterodyne systems*, which take advantage of Nyquist aliasing, bandpass filters are used to create sharp cut-offs on the edges of the band. Often it is necessary to introduce notch filters to reduce the effect of *radio frequency interference (RFI)*. Though the relative bandpass response can change with time, it is often slowly varying and can be consdiered stable for the duration of an observation.

The second component is a frequency-independent, time-varying gain due to the variation in the electronics and the observation pointing. This is often not modelled as it comes from a collection of random effects, but is accounted from during the calibration process discussed in later chapters.

### 7.3.4 Digitization

After signal conditioning in the analogue front-end, we are ready to capture the signal into a digital form so that we can perform the correlations. The term digitization is worth thinking about for a moment, it is the process of taking a continuous-in-time, analogue signal (which is described by real numbers), and converting it to a discrete in time and voltage signal (which are 'digits' described by integers). The last analogue component is the *analogue to digital converter (ADC)*. A simple form of analogue to digital conversion is a cascading resistor ladder which measures discrete steps in an analogue signal, each output of the ladder triggers a bit value of either 0 or 1, thus a digital signal is created. A typical radio astronomy ADC might have 8 bits, that is the signal can take on 256 values (-128 to 127). There is a resolution to these steps, which introduces digitization noise to the signal as the continuous signal is truncated to the closest step. You can imagine that more bits will reduce this noise, as there will be more steps and the resolution will be smaller. Or, more bits can be used to increase the dynamic range of the ADC. Say there is a strong RFI source with a weak sky signal, we want to be able to sample the sky signal at the enough resolution, but also we need to fully sample the RFI signal. If the maximum ADC value is smaller than the RFI signal then the RFI will *saturate* the ADC, causing the output of the ADC to sit at the highest or lowest value and no sky signal can be captured. This is another reason why strong RFI is often managed in the analogue chain. The choice of number of bits and step resolution depend on the RFI environment, the amplification of the sky signal, and also the bandwidth of the ADC.

***

* [7.4 Digital Electronics](7_4_digital.ipynb)